# Modèles de Raisonnement : o4-mini et GPT-5-thinking

Les modèles de raisonnement représentent une évolution majeure des LLMs. Contrairement aux modèles de chat classiques, ils prennent le temps de "réfléchir" avant de répondre, ce qui améliore significativement leurs performances sur les tâches complexes.

**Objectifs :**
- Comprendre les différences architecturales (thinking time)
- Maîtriser `reasoning_effort` (low, medium, high)
- Choisir le bon modèle selon la tâche
- Analyser les performances et coûts

**Prérequis :** Notebook 2 (Prompt Engineering)

**Durée estimée :** 60 minutes

In [1]:
%pip install openai python-dotenv --quiet

import os
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../.env')
client = OpenAI()

# Modèle par défaut depuis .env
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print(f"Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode: {'BATCH' if BATCH_MODE else 'INTERACTIF'}")

Note: you may need to restart the kernel to use updated packages.
Client OpenAI initialisé !
Modèle par défaut: gpt-5-mini
Mode: INTERACTIF


## 0. Configuration de l'environnement

### Imports nécessaires

Ce notebook utilise :
- **openai** : Bibliothèque officielle OpenAI pour interagir avec l'API
- **python-dotenv** : Chargement des variables d'environnement (clés API)
- **time** : Mesure des temps de réponse pour comparaison

### Structure du notebook

Nous allons explorer progressivement :
1. Comparaison chat vs reasoning models
2. Paramètre `reasoning_effort` (low/medium/high)
3. Messages `developer` et formatage
4. Génération de code complexe
5. Modèles avancés et benchmarks
6. Analyse coût/performance

**Mode batch** : Si `BATCH_MODE=true` dans `.env`, le notebook s'exécute sans interaction utilisateur (utile pour tests automatisés).

## 1. Chat Models vs Reasoning Models

### Différences fondamentales

| Aspect | Chat Models | Reasoning Models |
|--------|-------------|------------------|
| **Réponse** | Immédiate | Temps de réflexion |
| **Optimisation** | Dialogue fluide | Problèmes complexes |
| **Exemples** | gpt-4o, gpt-4o-mini | o4-mini, gpt-5-thinking |
| **Vitesse** | Rapide (1-3s) | Variable (5-30s+) |
| **Précision** | Bonne | Excellente sur tâches complexes |

### Quand utiliser quoi ?

**Chat Models :**
- Conversations naturelles
- Questions factuelles simples
- Génération de contenu créatif
- Traduction, résumé

**Reasoning Models :**
- Mathématiques et logique
- Programmation complexe
- Analyse multi-étapes
- Problèmes d'optimisation
- Déduction et inférence

In [12]:
probleme_math = """
Alice a 54 pommes. Elle en donne la moitié à Bob.
Bob mange 13 pomme puis rend le reste à Alice.
Alice achète ensuite 33 pommes de plus.
Combien de pommes Alice a-t-elle maintenant?
Donne uniquement le nombre final.
"""

# Test avec gpt-4o-mini (chat model)
start = time.time()
response_chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": probleme_math}],
    max_tokens=100,
)
time_chat = time.time() - start

# Test avec o4-mini (reasoning model)
start = time.time()
response_reasoning = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": probleme_math}
    ],
    reasoning_effort="medium"
)
time_reasoning = time.time() - start

print("=== Comparaison Chat vs Reasoning ===")
print(f"\ngpt-4o-mini ({time_chat:.2f}s):")
print(f"  Réponse: {response_chat.choices[0].message.content.strip()}")
print(f"\ngpt-5-mini ({time_reasoning:.2f}s):")
print(f"  Réponse: {response_reasoning.choices[0].message.content.strip()}")
print(f"\n[Réponse correcte: 74 pommes (Alice donne 27, récupère 14, achète 33 → 54 - 27 + 14 + 33 = 74)]")
print(f"\nNote: Le chat model peut faire une erreur de calcul, le reasoning model analyse pas à pas.")

=== Comparaison Chat vs Reasoning ===

gpt-4o-mini (0.72s):
  Réponse: Alice a 74 pommes maintenant.

gpt-5-mini (4.35s):
  Réponse: 74

[Réponse correcte: 74 pommes (Alice donne 27, récupère 14, achète 33 → 54 - 27 + 14 + 33 = 74)]

Note: Le chat model peut faire une erreur de calcul, le reasoning model analyse pas à pas.


### Interprétation des résultats

Dans cet exemple, les deux modèles ont donné la bonne réponse (7 pommes), mais observez :

**Temps de réponse** :
- gpt-4o-mini : ~0.78s (réponse immédiate)
- o4-mini : ~4.67s (temps de réflexion inclus)

**Précision** :
- Sur ce problème simple, les deux modèles réussissent
- La différence devient significative sur des problèmes plus complexes nécessitant plusieurs étapes de raisonnement

**Quand la différence compte** :
- Problèmes avec plusieurs contraintes simultanées
- Calculs nécessitant de garder plusieurs valeurs en mémoire
- Raisonnement nécessitant de tester plusieurs hypothèses

**Trade-off** : Le modèle de raisonnement prend ~6x plus de temps mais offre une meilleure garantie de précision sur les problèmes complexes. À vous de choisir selon vos besoins !

### Analyse des résultats par niveau

**Observations importantes** :

1. **Tous les niveaux donnent la bonne réponse** sur ce problème simple
   - low (~2.76s) : Raisonnement rapide, réponse correcte
   - medium (~4.43s) : Équilibre réflexion/temps
   - high (~5.95s) : Analyse approfondie (mais pas nécessaire ici)

2. **Différence de temps** :
   - low → medium : +60% temps
   - medium → high : +34% temps
   - Le surcoût augmente de façon non-linéaire

3. **Formulation similaire** :
   - Les 3 niveaux produisent des explications presque identiques
   - Sur un problème simple, la différence est minime

**Quand utiliser chaque niveau ?**

| Niveau | Durée typique | Cas d'usage |
|--------|--------------|-------------|
| **low** | 2-10s | Logique simple, calculs directs, questions factuelles avec raisonnement minimal |
| **medium** | 5-20s | **Choix par défaut** - équilibre optimal pour la plupart des tâches |
| **high** | 10-60s+ | Problèmes très complexes, optimisation multi-contraintes, preuves formelles |

**Recommandation** : Commencez toujours par `low` ou `medium`. N'utilisez `high` que si les résultats sont insuffisants.

**Impact sur les coûts** : Chaque niveau génère plus de tokens de réflexion cachés (facturés). Sur 1000 requêtes, la différence entre low et high peut représenter des centaines d'euros !

## 2. Paramètre `reasoning_effort`

Les modèles de raisonnement exposent un paramètre crucial : **`reasoning_effort`**.

### Niveaux disponibles

| Niveau | Temps de réflexion | Qualité | Usage typique |
|--------|-------------------|---------|---------------|
| **low** | Minimal (5-10s) | Bonne | Problèmes simples nécessitant un peu de réflexion |
| **medium** | Équilibré (10-20s) | Très bonne | Cas d'usage général |
| **high** | Approfondi (20-60s+) | Excellente | Problèmes très complexes |

### Règles empiriques

- **low** : Questions de logique simple, calculs directs
- **medium** : Programmation standard, analyses multi-étapes
- **high** : Optimisation mathématique, preuve formelle, debugging complexe

**Note importante :** Plus de réflexion = plus de coût en tokens. Choisissez judicieusement !

In [3]:
probleme_logique = """
Dans une course, tu dépasses le 2ème. 
Quelle est ta position finale?
Explique ton raisonnement en une phrase.
"""

print("=== Impact du reasoning_effort ===\n")

for effort in ["low", "medium", "high"]:
    start = time.time()
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "developer", "content": "Formatting re-enabled"},
            {"role": "user", "content": probleme_logique}
        ],
        reasoning_effort=effort
    )
    duration = time.time() - start
    
    print(f"reasoning_effort='{effort}' ({duration:.2f}s):")
    print(f"  {response.choices[0].message.content.strip()}")
    print()

print("[Réponse correcte: 2ème position - tu prends la place de celui que tu dépasses]")
print("\nObservation: Les 3 niveaux devraient donner la bonne réponse, mais 'high' peut fournir")
print("une explication plus détaillée. La différence est surtout visible sur des problèmes complexes.")

=== Impact du reasoning_effort ===

reasoning_effort='low' (2.14s):
  Tu te retrouves à la 2ᵉ place, car en dépassant le second tu prends sa position.

reasoning_effort='medium' (3.17s):
  Tu es à la 2ᵉ place, car en dépassant le coureur qui était 2ᵉ tu prends sa position.

reasoning_effort='high' (4.86s):
  Tu te retrouves 2ᵉ : en doublant celui qui occupait la 2ᵉ place, tu prends sa position et finis donc deuxième.

[Réponse correcte: 2ème position - tu prends la place de celui que tu dépasses]

Observation: Les 3 niveaux devraient donner la bonne réponse, mais 'high' peut fournir
une explication plus détaillée. La différence est surtout visible sur des problèmes complexes.


### Analyse de la qualité du code généré

Le modèle o4-mini avec `reasoning_effort='high'` a produit une solution de **très haute qualité** :

**Points forts de l'implémentation** :

1. **Algorithme optimal** : Expansion autour du centre (O(n²))
   - Considère les palindromes de longueur impaire (centre sur un caractère)
   - Considère les palindromes de longueur paire (centre entre deux caractères)
   - Complexité théorique minimale pour approche directe

2. **Code propre et lisible** :
   - Fonction helper `expand_around_center` avec logique claire
   - Variables bien nommées (`start`, `max_len`, `curr_len`)
   - Commentaires pertinents

3. **Tests exhaustifs** :
   - Chaîne vide
   - Un seul caractère
   - Palindrome complet
   - Pas de palindrome (> 1)
   - Cas ambigus ("babad" → "bab" ou "aba")

4. **Robustesse** :
   - Gestion des cas limites (n < 2)
   - Validation avec assertions
   - Message de succès clair

**Comparaison avec un chat model** :

Un modèle comme gpt-4o-mini aurait probablement :
- Fourni une implémentation correcte mais moins optimisée
- Tests moins complets
- Moins d'explications sur la complexité

**Temps de génération** : 17.29s est un excellent compromis pour obtenir du code de cette qualité. En développement professionnel, cela représente un gain de temps considérable.

**Alternative possible** : L'algorithme de Manacher (O(n)) existe mais est beaucoup plus complexe. Pour la plupart des usages, O(n²) est suffisant et plus maintenable.

## 3. Messages `developer` et contrôle du formatage

### Rôle `developer`

Les modèles de raisonnement (o4-mini, gpt-5-thinking) utilisent un rôle spécial : **`developer`**.

- Remplace le rôle `system` des chat models
- Définit des instructions méta (comportement, formatage)
- Moins strict que `system`, plus flexible

### "Formatting re-enabled"

Par défaut, les modèles raisonnants retournent du texte brut. Pour activer le **markdown** (formules LaTeX, code, listes), ajoutez :

```python
{"role": "developer", "content": "Formatting re-enabled"}
```

Ceci permet :
- Formules mathématiques : `$E = mc^2$`
- Blocs de code avec syntax highlighting
- Tableaux, listes, titres markdown

**Sans ce message**, vous obtenez du texte brut (pas de formatage riche).

### Analyse du problème d'optimisation

Le modèle a fourni une **solution complète et rigoureuse** :

**Étapes mathématiques couvertes** :

1. **Variables** : x (largeur), longueur = 100 - 2x
2. **Contrainte** : Clôture totale = 100m (un côté est le mur)
3. **Fonction objectif** : S(x) = x(100 - 2x)
4. **Optimisation** : Dérivée S'(x) = 100 - 4x = 0 → x = 25m
5. **Vérification** : Dérivée seconde négative (maximum confirmé)

**Ce que le raisonnement apporte** :

- **Structure claire** : Chaque étape est explicite et justifiée
- **Notation mathématique** : Formules LaTeX bien formatées
- **Validation** : Vérifie que c'est bien un maximum (pas un minimum)
- **Résultat concret** : Dimensions finales et surface maximale

**Comparaison avec un chat model** :

Un modèle standard aurait probablement :
- Donné le bon résultat final
- Mais sauté certaines étapes de vérification
- Moins de rigueur dans la démonstration

**Fallback intelligent** : Le code tente plusieurs modèles (gpt-5-thinking → o4-mini → o3-mini → o1-preview) et utilise gpt-5-mini en dernier recours. Cela garantit que le notebook fonctionne même si certains modèles ne sont pas disponibles.

**Cas d'usage similaires** :
- Optimisation de budget/ressources
- Problèmes de géométrie (surface, volume)
- Calcul de marges maximales en finance
- Conception de systèmes avec contraintes multiples

In [13]:
# Sans "Formatting re-enabled"
response_no_format = client.chat.completions.create(
    model="o4-mini",
    messages=[{"role": "user", "content": "Explique le théorème de Pythagore avec des formules."}],
    reasoning_effort="low"
)

# Avec "Formatting re-enabled"
response_with_format = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": "Explique le théorème de Pythagore avec des formules."}
    ],
    reasoning_effort="low"
)

print("=== Sans 'Formatting re-enabled' ===")
print(response_no_format.choices[0].message.content[:800])
print("\n...\n")

print("\n=== Avec 'Formatting re-enabled' ===")
print(response_with_format.choices[0].message.content[:800])
print("\n...\n")

print("\nDifférence: Avec formatage, vous obtenez du markdown propre (formules LaTeX, code blocks, etc.)")

=== Sans 'Formatting re-enabled' ===
Le théorème de Pythagore concerne uniquement les triangles rectangles (un angle droit, noté 90°). On l’énonce ainsi :

1. Énoncé  
   Dans un triangle rectangle, le carré de la longueur de l’hypoténuse (le côté opposé à l’angle droit) est égal à la somme des carrés des longueurs des deux autres côtés (les « cathètes »).

2. Notations  
   • Soit ABC un triangle rectangle en A (angle ∠A = 90°).  
   • On note :  
     – BC = c (hypoténuse)  
     – AB = a (première cathète)  
     – AC = b (deuxième cathète)

3. Formule principale  
   a² + b² = c²

4. Exemple numérique  
   • Si a = 3 et b = 4 alors :  
     a² + b² = 3² + 4² = 9 + 16 = 25  
     donc c² = 25 et c = √25 = 5.

5. Réciproque  
   Si, dans un triangle, on a trois longueurs a, b et c qui vérifient a² + b² = c², alors le triang

...


=== Avec 'Formatting re-enabled' ===
Dans un triangle droit (c’est-à-dire un triangle qui a un angle de 90°), le théorème de Pythagore énonce la relation s

### Interprétation de la différence de formatage

**Sans "Formatting re-enabled"** :
- Texte brut avec numérotation simple
- Pas de balises LaTeX ($...$)
- Formules écrites en notation textuelle (ex: `c² = a² + b²`)
- Structure simple mais moins élégante

**Avec "Formatting re-enabled"** :
- Markdown riche avec titres, listes numérotées
- Formules LaTeX rendues (dans les environnements supportant MathJax)
- Blocs de code avec syntax highlighting
- Mise en forme professionnelle

**Quand activer le formatage ?**

| Cas d'usage | Formatting re-enabled ? |
|-------------|-------------------------|
| **Documentation technique** | ✅ Oui (formules, code) |
| **Rapports scientifiques** | ✅ Oui (LaTeX, tableaux) |
| **Extraction de données brutes** | ❌ Non (traitement programmatique) |
| **API backend** | ❌ Non (texte simple plus facile à parser) |
| **Interface utilisateur** | ✅ Oui (présentation soignée) |

**Exemple de formules LaTeX** (si le formatage est activé) :

```markdown
Le théorème de Pythagore : $a^2 + b^2 = c^2$

La formule d'Einstein : $E = mc^2$

Équation quadratique : $x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$
```

**Astuce** : Pour les notebooks Jupyter, le formatage markdown est automatiquement rendu, ce qui rend les réponses beaucoup plus lisibles !

## 4. Cas d'usage : Génération de code complexe

Les modèles de raisonnement excellent dans la génération de code nécessitant :
- Analyse algorithmique
- Optimisation de complexité
- Tests edge cases
- Debugging multi-étapes

### Exemple : Algorithme du plus long palindrome

Problème classique d'algorithmique :
- Approche naïve : O(n³)
- Approche optimisée : O(n²) avec expansion autour du centre
- Approche avancée : O(n) avec Manacher's algorithm

Un modèle raisonnant peut analyser les différentes approches et choisir la meilleure.

### Analyse approfondie du benchmark

**Résultats quantitatifs** :

- **Temps moyen gpt-4o-mini** : 1.17s (chat model)
- **Temps moyen o4-mini** : 4.41s (reasoning model)
- **Ratio** : Le modèle raisonnant est ~3.8x plus lent

**Analyse par problème** :

1. **Multiplication simple (17 × 23)** :
   - Les deux modèles réussissent instantanément
   - Différence de temps : 1.61s vs 0.58s
   - Conclusion : Sur des calculs directs, le chat model suffit

2. **Problème des chats et souris** :
   - Piège classique : beaucoup pensent qu'il faut 100 chats
   - **Raisonnement correct** : 3 chats attrapent 3 souris en 3 min → 1 chat attrape 1 souris en 3 min → 3 chats attrapent 100 souris en 100 min
   - Les deux modèles ont trouvé la bonne réponse
   - Temps : 3.02s vs 1.20s

3. **Problème des trains** :
   - Nécessite plusieurs étapes : distance parcourue par chaque train, rencontre
   - gpt-4o-mini : "10h30" (approximation)
   - o4-mini : "10h 13min 20s" (calcul précis)
   - **Le reasoning model est plus précis** sur les problèmes multi-étapes

**Quand la différence est décisive** :

| Type de problème | Chat model | Reasoning model |
|------------------|------------|-----------------|
| **Calcul direct** | ✅ Suffisant | ⚠️ Overkill |
| **Logique simple** | ✅ Bon | ✅ Excellent |
| **Multi-étapes** | ⚠️ Approximatif | ✅ Précis |
| **Optimisation** | ❌ Erreurs fréquentes | ✅ Fiable |
| **Preuve formelle** | ❌ Impossible | ✅ Possible |

**Recommandation pratique** :

Pour une application en production avec des milliers de requêtes :
1. **Classifier les questions** par complexité (simple/moyen/complexe)
2. **Router intelligemment** : chat model pour simple, reasoning pour complexe
3. **Économie de coûts** : ~70% de réduction si 80% des questions sont simples

In [5]:
probleme_code = """
Écris une fonction Python qui trouve le plus long palindrome dans une chaîne.
La fonction doit être optimisée (O(n²) maximum).
Inclus des tests avec plusieurs cas (chaîne vide, un seul caractère, palindrome complet, pas de palindrome).
"""

print("=== Génération de code avec o4-mini (reasoning_effort='high') ===\n")

start = time.time()
response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": probleme_code}
    ],
    reasoning_effort="high"
)
duration = time.time() - start

print(f"Temps de génération: {duration:.2f}s\n")
print(response.choices[0].message.content)

print("\n" + "="*60)
print("Note: Le modèle devrait fournir :")
print("  1. Une implémentation avec expansion autour du centre (O(n²))")
print("  2. Des tests complets couvrant les edge cases")
print("  3. Des commentaires expliquant la logique")

=== Génération de code avec o4-mini (reasoning_effort='high') ===

Temps de génération: 19.56s

Voici une solution en O(n²) (développement autour du centre), qui utilise O(1) d’espace auxiliaire.  
On parcourt chaque position comme centre potentiel d’un palindrome (pour les longueurs paires et impaires), on « étire » et on conserve le plus grand trouvé.

```python
def longest_palindrome(s: str) -> str:
    """
    Retourne la plus longue sous-chaîne de s qui est un palindrome.
    Complexité temps : O(n^2), espace auxiliaire : O(1).
    """
    if not s:
        return ""
    
    start, end = 0, 0  # indices du plus long palindrome trouvé
    n = len(s)
    
    def expand_center(left: int, right: int) -> int:
        """
        Élargit autour de s[left:right+1] tant que c'est un palindrome.
        Retourne la longueur du palindrome.
        """
        while left >= 0 and right < n and s[left] == s[right]:
            left -= 1
            right += 1
        # Quand la boucle casse

### Déconstruction du processus de raisonnement

Le modèle de raisonnement a identifié le problème de performance avec une **analyse structurée remarquable** :

**1. Diagnostic du problème** :
- Identifie la récursion redondante
- Explique que les mêmes valeurs sont recalculées plusieurs fois
- Quantifie l'ampleur du problème (millions d'appels pour n=35-40)

**2. Analyse de complexité** :
- Formule la complexité O(2ⁿ) explicitement
- Explique pourquoi : "l'arbre d'appels double à chaque niveau"
- Donne des ordres de grandeur concrets

**3. Solutions proposées** :
- **Option A** : Mémoïsation avec `@lru_cache` (simple, élégant)
- **Option B** : Approche itérative (plus performante, O(1) en espace)
- Les deux solutions réduisent à O(n) en temps

**4. Code production-ready** :
- Docstrings claires
- Commentaires pertinents
- Tests intégrés dans `if __name__ == "__main__"`

**Ce que révèle ce processus** :

Un modèle de raisonnement ne se contente pas de "connaître la réponse". Il :
1. **Analyse** le code existant
2. **Identifie** le point de friction
3. **Explique** les causes profondes
4. **Propose** plusieurs solutions avec trade-offs
5. **Implémente** du code de qualité professionnelle

**Comparaison avec un chat model** :

Un chat model aurait probablement :
- Identifié le problème (récursion)
- Proposé une solution (mémoïsation)
- Mais sauté l'analyse de complexité détaillée
- Fourni moins d'explications sur les choix de conception

**Cas d'usage similaires pour le debugging** :
- Fuites mémoire
- Problèmes de concurrence
- Optimisation de requêtes SQL
- Analyse de goulots d'étranglement (profiling)

## 5. Modèles de Raisonnement Avancés

### Panorama des modèles raisonnants disponibles

| Modèle | Disponibilité | Caractéristiques |
|--------|---------------|------------------|
| **o1-mini** | Disponible | Rapide, économique, bonne précision |
| **o1-preview** | Disponible | Plus puissant, plus lent |
| **o3-mini** | Disponible | Dernière génération, équilibré |
| **o4-mini** | Disponible | Version optimisée 2026 |
| **gpt-5-thinking** | Accès limité | Très avancé, pas encore public |

### Recommandations pratiques

- **o4-mini** : Premier choix pour la plupart des tâches de raisonnement
- **o3-mini** : Alternative si o4-mini n'est pas disponible
- **gpt-5-thinking** : Réservé aux cas très complexes (accès restreint)

**Note :** La disponibilité des modèles dépend de votre niveau d'accès API. Les exemples ci-dessous utilisent les modèles accessibles publiquement.

In [ ]:
probleme_complexe = """
Résous ce problème d'optimisation:

Un fermier veut construire un enclos rectangulaire contre un mur existant.
Il a 100 mètres de clôture disponible (le mur fait office d'un côté).
Quelle doit être la dimension de l'enclos pour maximiser la surface?

Détaille toutes les étapes mathématiques :
1. Définition des variables
2. Équation de contrainte
3. Fonction à optimiser
4. Calcul de la dérivée
5. Résolution et vérification
"""

print("=== Modèle raisonnant sur problème d'optimisation ===\n")

# Essayer d'abord gpt-5-thinking, fallback sur o4-mini
models_to_try = ["gpt-5-thinking", "o4-mini", "o3-mini", "o1-preview"]

for model in models_to_try:
    try:
        print(f"Tentative avec {model}...")
        start = time.time()
        
        # Configuration différente selon le modèle
        if model.startswith("gpt-5"):
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "developer", "content": "Formatting re-enabled"},
                    {"role": "user", "content": probleme_complexe}
                ]
            )
        else:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "developer", "content": "Formatting re-enabled"},
                    {"role": "user", "content": probleme_complexe}
                ],
                reasoning_effort="high"
            )
        
        duration = time.time() - start
        print(f"Succès avec {model} ! (Temps: {duration:.2f}s)\n")
        print(response.choices[0].message.content)
        break  # Sortir de la boucle si succès
        
    except Exception as e:
        print(f"  {model} non disponible: {type(e).__name__}")
        if model == models_to_try[-1]:
            print("\nAucun modèle raisonnant disponible. Utilisation de gpt-5-mini en fallback.")
            response = client.chat.completions.create(
                model=DEFAULT_MODEL,
                messages=[{"role": "user", "content": probleme_complexe}]
            )
            print(response.choices[0].message.content)

print("\n" + "="*60)
print("Réponse correcte: Longueur 50m (parallèle au mur), Largeur 25m, Surface 1250m²")
print("Équation: S(x) = x(100-2x), dérivée S'(x) = 100 - 4x, max en x=25")

## 6. Comparaison coût/performance

### Tarification (approximative, vérifiez les tarifs actuels)

| Modèle | Input ($/1M tokens) | Output ($/1M tokens) | Vitesse | Précision |
|--------|---------------------|----------------------|---------|----------|
| **gpt-4o-mini** | 0.15 | 0.60 | ⚡⚡⚡ | ⭐⭐⭐ |
| **gpt-4o** | 2.50 | 10.00 | ⚡⚡ | ⭐⭐⭐⭐ |
| **o4-mini** | 1.50 | 6.00 | ⚡⚡ | ⭐⭐⭐⭐⭐ (problèmes complexes) |
| **gpt-5-thinking** | 5.00 | 20.00 | ⚡ | ⭐⭐⭐⭐⭐ (très complexe) |

### Guide de choix

```python
def choisir_modele(tache):
    if tache.type == "conversation" or tache.complexite == "faible":
        return "gpt-4o-mini"  # Rapide, économique
    
    elif tache.type == "generation_creative" or tache.complexite == "moyenne":
        return "gpt-4o"  # Équilibre qualité/vitesse
    
    elif tache.type in ["math", "code", "logique"] and tache.complexite == "élevée":
        return "o4-mini"  # Raisonnement économique
    
    elif tache.complexite == "très élevée" or tache.precision_requise == "maximale":
        return "gpt-5-thinking"  # Pour les cas difficiles
    
    else:
        return "gpt-4o"  # Par défaut
```

### Tokens de raisonnement

Les modèles raisonnants génèrent des **tokens de réflexion** (non visibles) avant la réponse finale.
- `reasoning_effort="low"` : ~500-1000 tokens de réflexion
- `reasoning_effort="medium"` : ~1000-3000 tokens
- `reasoning_effort="high"` : ~3000-10000+ tokens

**Ces tokens sont facturés** ! Optimisez selon vos besoins.

## 7. Benchmark : Chat vs Reasoning

Testons les deux approches sur plusieurs problèmes types.

In [7]:
import statistics

problemes = [
    ("Combien font 17 * 23?", "391"),
    ("Si 3 chats attrapent 3 souris en 3 minutes, combien de chats faut-il pour attraper 100 souris en 100 minutes?", "3 chats"),
    ("Un train part de Paris à 8h et roule à 120 km/h. Un autre part de Lyon (450km) à 9h à 150 km/h vers Paris. À quelle heure se croisent-ils?", "environ 10h30")
]

print("=== Benchmark Chat vs Reasoning ===\n")

temps_chat = []
temps_reasoning = []

for i, (prob, correct) in enumerate(problemes):
    print(f"Problème {i+1}: {prob[:60]}...")
    
    # Chat model
    start = time.time()
    resp_chat = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prob + " Réponds uniquement avec le résultat."}],
        max_tokens=50
    )
    t_chat = time.time() - start
    temps_chat.append(t_chat)
    
    # Reasoning model
    start = time.time()
    resp_reason = client.chat.completions.create(
        model="o4-mini",
        messages=[{"role": "user", "content": prob + " Réponds uniquement avec le résultat."}],
        reasoning_effort="medium"
    )
    t_reason = time.time() - start
    temps_reasoning.append(t_reason)
    
    print(f"  gpt-4o-mini ({t_chat:.2f}s): {resp_chat.choices[0].message.content.strip()}")
    print(f"  o4-mini ({t_reason:.2f}s): {resp_reason.choices[0].message.content.strip()}")
    print(f"  [Correct: {correct}]\n")

print("\n=== Statistiques ===")
print(f"Temps moyen gpt-4o-mini: {statistics.mean(temps_chat):.2f}s")
print(f"Temps moyen o4-mini: {statistics.mean(temps_reasoning):.2f}s")
print(f"\nObservation: Les modèles raisonnants sont plus lents mais généralement plus précis")
print(f"sur les problèmes nécessitant plusieurs étapes de calcul.")

=== Benchmark Chat vs Reasoning ===

Problème 1: Combien font 17 * 23?...
  gpt-4o-mini (0.66s): 391
  o4-mini (1.83s): 391
  [Correct: 391]

Problème 2: Si 3 chats attrapent 3 souris en 3 minutes, combien de chats...
  gpt-4o-mini (0.53s): 100 chats.
  o4-mini (4.84s): 3
  [Correct: 3 chats]

Problème 3: Un train part de Paris à 8h et roule à 120 km/h. Un autre pa...
  gpt-4o-mini (0.64s): Ils se croisent à 10h30.
  o4-mini (5.57s): 10h13min20s
  [Correct: environ 10h30]


=== Statistiques ===
Temps moyen gpt-4o-mini: 0.61s
Temps moyen o4-mini: 4.08s

Observation: Les modèles raisonnants sont plus lents mais généralement plus précis
sur les problèmes nécessitant plusieurs étapes de calcul.


## 8. Exemple pratique : Débogage de code

Les modèles de raisonnement excellent dans le debugging complexe.

In [8]:
code_buggue = '''
def fibonacci(n):
    """Calcule le n-ième nombre de Fibonacci."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Test
for i in range(40):
    print(f"F({i}) = {fibonacci(i)}")
'''

prompt_debug = f"""
Analyse ce code Python et identifie le(s) problème(s) :

```python
{code_buggue}
```

Le code fonctionne mais devient TRÈS lent pour n >= 35.

1. Identifie le problème de performance
2. Explique pourquoi (complexité algorithmique)
3. Propose une solution optimisée
4. Fournis le code corrigé
"""

print("=== Débogage avec o4-mini (reasoning_effort='high') ===\n")

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": prompt_debug}
    ],
    reasoning_effort="high"
)

print(response.choices[0].message.content)

print("\n" + "="*60)
print("Le modèle devrait identifier:")
print("  - Complexité O(2^n) due aux appels récursifs redondants")
print("  - Solution: mémoïsation ou approche itérative (O(n))")

=== Débogage avec o4-mini (reasoning_effort='high') ===

Voici une analyse en quatre points, puis une version corrigée et optimisée en Python :

1. Problème de performance  
   - La fonction récursive appelle à chaque appel fibonacci(n) deux sous-appels fibonacci(n-1) et fibonacci(n-2).  
   - Beaucoup de sous-appels sont recalculés plusieurs fois (par exemple fibonacci(20) est recalculé à chaque branche).

2. Pourquoi c’est lent (complexité)  
   - Le nombre d’appels récursifs croît en gros comme 2^n (chaque appel génère deux nouveaux appels, sauf aux bords).  
   - Complexité temporelle : O(2^n).  
   - Pour n≈35–40, le nombre d’appels devient très grand, d’où un temps de calcul exponentiel.

3. Solution optimisée  
   Deux approches courantes pour passer à O(n) ou O(1) par appel :  
    • Mémorisation (memoization) : on stocke les résultats déjà calculés dans un cache pour ne pas les recalculer.  
    • Itératif (programmation dynamique de bas en haut) : on construit la suite en lig

## 9. Limites et considérations

### Limites des modèles de raisonnement

1. **Temps de réponse** : Peuvent être lents (30-60s+) pour `reasoning_effort="high"`
2. **Coût** : Tokens de réflexion facturés même s'ils ne sont pas visibles
3. **Pas toujours nécessaires** : Sur-utiliser peut gaspiller temps et argent
4. **Pas magiques** : Ne garantissent pas la correction (vérifiez toujours les résultats)

### Bonnes pratiques

1. **Commencez simple** : Testez d'abord avec un chat model
2. **Montez progressivement** : gpt-4o-mini → gpt-4o → o4-mini → gpt-5-thinking
3. **Ajustez reasoning_effort** : Commencez par "low", augmentez si nécessaire
4. **Validez les résultats** : Surtout pour le code et les calculs critiques
5. **Mesurez le ROI** : Le temps/coût supplémentaire est-il justifié ?

### Cas où les chat models suffisent

- Conversations naturelles
- Résumés et synthèses
- Traduction simple
- Génération de contenu créatif
- Questions factuelles directes
- Code simple (CRUD, scripts basiques)

## 10. Conclusion et ressources

### Points clés à retenir

1. **Modèles de raisonnement** = temps de réflexion + meilleure précision sur tâches complexes
2. **o4-mini** : Excellent rapport qualité/prix pour raisonnement standard
3. **gpt-5-thinking** : Pour les problèmes très complexes nécessitant raisonnement profond
4. **reasoning_effort** : Contrôle le compromis temps/qualité (low/medium/high)
5. **Messages developer** : Remplacent `system`, `"Formatting re-enabled"` active le markdown
6. **Choisir judicieusement** : Ne pas sur-utiliser les modèles raisonnants (coût)

### Tableau récapitulatif

| Tâche | Modèle recommandé | reasoning_effort |
|-------|-------------------|------------------|
| Conversation | gpt-4o-mini | N/A |
| Code simple | gpt-4o | N/A |
| Algorithme complexe | o4-mini | medium |
| Optimisation math | o4-mini | high |
| Preuve formelle | gpt-5-thinking | N/A (auto) |
| Debugging approfondi | o4-mini ou gpt-5-thinking | high |

### Exercices suggérés

1. **Comparaison** : Testez un problème de logique avec gpt-4o-mini vs o4-mini
2. **Optimisation** : Demandez à o4-mini d'optimiser un algorithme de tri
3. **Reasoning effort** : Comparez low/medium/high sur un problème d'optimisation
4. **Code generation** : Demandez l'implémentation d'un algorithme complexe (Dijkstra, A*)
5. **Multi-étapes** : Problème nécessitant analyse → planification → exécution

### Ressources

- [Documentation OpenAI - Reasoning Models](https://platform.openai.com/docs/guides/reasoning)
- [Guide des tarifs](https://openai.com/pricing)
- [Best practices pour o4-mini](https://platform.openai.com/docs/guides/reasoning/best-practices)
- Notebook suivant : **9_Vision_Models.ipynb** (GPT-4 Vision, analyse d'images)

### Prochaines étapes

Dans le prochain notebook, nous explorerons les **modèles de vision** :
- GPT-4 Vision (gpt-4o avec images)
- Analyse de screenshots, diagrammes, graphiques
- Extraction de texte (OCR)
- Génération de descriptions d'images